In [1]:
# This script will map hub locations for Wilmington before vs. after adding in new potential hub locations

import os
import pandas as pd
import numpy as np

import geopandas as gpd
import pandas as pd
import numpy as np
from statistics import mean

import itertools

import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import plotly
import shapely
import folium
import plotly.graph_objects as go # or plotly.express as px
import seaborn as sns

from data_cleaning_cmm import (dist_to_site_wilmington_walk_transit_df,
                               dist_to_site_wilmington_old_sites_df)


In [2]:
# Load sites data
ca_albers_nad83 = 'NAD_1983_California_Teale_Albers_FtUS'
nad83 = 'EPSG:4269'
wgs84 = 'EPSG:4326'

# Building candidate sites GeoDataFrame
sites_path = os.path.join(os.getcwd(), 'data', 'candidate_site_campuses_2021-11-17', 'candidate_sites_campuses.csv')
sites_df_raw = pd.read_csv(sites_path)
sites_df_raw = sites_df_raw.loc[sites_df_raw['cat_site'] != 'X', ['id_site', 'type_site', 'cat_site', 'SQFT_ROOF', 'LON', 'LAT']]
sites_geom = gpd.points_from_xy(sites_df_raw.LON, sites_df_raw.LAT, crs = nad83)
sites_gdf = gpd.GeoDataFrame(sites_df_raw, geometry = sites_geom, crs = nad83)

sites_path_wilmington = os.path.join(os.getcwd(), 'data', 'candidate_site_campuses_2021-11-17', 'estimated_square_footages.csv')
sites_df_raw_wilmington = pd.read_csv(sites_path_wilmington)
sites_df_raw_wilmington["id_site"] = np.arange(len(sites_df_raw_wilmington))
sites_df_raw_wilmington = sites_df_raw_wilmington.loc[sites_df_raw_wilmington['cat_site'] != 'X', ['id_site', 'type_site', 'cat_site', 'SQFT_ROOF', 'LON', 'LAT']]
sites_geom_wilmington = gpd.points_from_xy(sites_df_raw_wilmington.LON, sites_df_raw_wilmington.LAT, crs = nad83)
sites_gdf_wilmington = gpd.GeoDataFrame(sites_df_raw_wilmington, geometry = sites_geom_wilmington, crs = nad83)


In [3]:
# restrict sites data to sites used in Wilmington
sites_gdf_adj = sites_gdf.copy()
sites_gdf_adj = sites_gdf_adj.loc[sites_gdf_adj['id_site'].isin(dist_to_site_wilmington_old_sites_df.columns)]

sites_gdf_wilmington_adj = sites_gdf_wilmington.copy()
sites_gdf_wilmington_adj = sites_gdf_wilmington_adj.loc[sites_gdf_wilmington_adj['id_site'].isin([int(site) for site in dist_to_site_wilmington_walk_transit_df.columns])]

In [4]:
band_categories = ['CC', 'W', 'SCHOOL']
band_colors_list = ['Community Center', 'Place of Worship', 'School']
#band_colors_list = ['Red', 'Green', 'Blue', 'Blue']
band_dict = dict(zip(band_categories,band_colors_list)) #set up band to color substitution dict
sites_gdf_wilmington_adj['Site Type'] = sites_gdf_wilmington_adj['cat_site'].replace(to_replace=band_dict)

In [14]:
sites_gdf_wilmington_adj

,id_site,type_site,cat_site,SQFT_ROOF,LON,LAT,geometry,Site Type
2490,2490,Elementary,SCHOOL,49537.166504,-118.265820,33.776400,POINT (-118.26582 33.77640),School
2668,2668,Elementary,SCHOOL,114386.644477,-118.261800,33.787420,POINT (-118.26180 33.78742),School
2760,2760,Elementary,SCHOOL,5782.532950,-118.263589,33.799820,POINT (-118.26359 33.79982),School
2776,2776,High,SCHOOL,264878.143172,-118.261380,33.794520,POINT (-118.26138 33.79452),School
2809,2809,Continuation,SCHOOL,65165.501322,-118.263970,33.791910,POINT (-118.26397 33.79191),School
...,...,...,...,...,...,...,...,...
11227,11227,X,W,1796.698636,-118.251360,33.793401,POINT (-118.25136 33.79340),Place of Worship
11228,11228,X,W,2515.421071,-118.246959,33.790341,POINT (-118.24696 33.79034),Place of Worship
11229,11229,X,W,2455.364266,-118.244633,33.787944,POINT (-118.24463 33.78794),Place of Worship
11230,11230,X,W,2648.677846,-118.246970,33.789975,POINT (-118.24697 33.78997),Place of Worship


In [12]:
color_map = {"Community Center": '#FF9252', 'Place of Worship': '#B55AD6', 'School': '#00B2CA'}
fig = px.scatter_mapbox(sites_gdf_wilmington_adj,
                        lat='LAT', 
                        lon='LON',  
                        color='Site Type', 
                        color_discrete_map=color_map,
                        category_orders={'Site Type':('Community Center','Place of Worship','School')},
                        mapbox_style="carto-positron", 
                        #mapbox_style = "open-street-map",
                        zoom=13, 
                        title="Local Model Wilmington Sites",
                        center=dict(lat=33.7861, 
                                     lon = -118.264),
                        width=1000, height=1000
                       )
fig.show()
plotly.io.write_image(fig, 'results/figures/fig_1d.svg', format='svg', scale=1, width=1000, height=1000)

In [10]:
band_categories = ['CC', 'W', 'Sec', 'Pri']
band_colors_list = ['Community Center', 'Place of Worship', 'School', 'School']
#band_colors_list = ['Red', 'Green', 'Blue', 'Blue']
band_dict = dict(zip(band_categories,band_colors_list)) #set up band to color substitution dict
sites_gdf_adj['Site Type'] = sites_gdf_adj['cat_site'].replace(to_replace=band_dict)

In [20]:
color_map = {"Community Center": '#FF9252', 'Place of Worship': '#B55AD6', 'School': '#00B2CA'}
fig = px.scatter_mapbox(sites_gdf_adj,
                        lat='LAT', 
                        lon='LON',  
                        color='Site Type',
                        color_discrete_map=color_map,
                        category_orders={'Site Type':('Community Center','Place of Worship','School')},
                        mapbox_style="carto-positron", 
                        zoom=13, 
                        title="Statewide Model Wilmington Sites",
                        center=dict(lat=33.7861, 
                                     lon = -118.264),
                        width=1000, height=1000
                       )
fig.show()
plotly.io.write_image(fig, 'results/figures/fig_1c.svg', format='svg', scale=1, width=1000, height=1000)